In [ ]:
%%capture
import os, re
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Nếu chạy Colab notebooks thì sẽ install thư viện theo các phiên bản sau
    import torch; v = re.match(r"[0-9\.]{3,}", str(torch.__version__)).group(0)
    xformers = "xformers==" + ("0.0.32.post2" if v == "2.8.0" else "0.0.29.post3")
    !pip install --no-deps bitsandbytes accelerate {xformers} peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1,<4.0.0" "huggingface_hub>=0.34.0" hf_transfer
    !pip install --no-deps unsloth
!pip install transformers==4.56.2
!pip install --no-deps trl==0.22.2

In [ ]:
from unsloth import FastLanguageModel, is_bfloat16_supported
import torch

from trl import SFTTrainer
from transformers import TrainingArguments

# **Load dataset**

+ **Instruction**: mô tả nhiệm vụ mà model cần hoàn thành - đây là hướng dẫn hoặc yêu cầu cụ thể mà chúng ta muốn model thực hiện
+ **Input**: dữ liệu đầu vào bổ sung thêm ngữ cảnh để model hoàn thành instruction
+ **Output**: Đây là phản hồi hoặc là kết quả ta mong muốn model trả về

In [ ]:
# Alpaca
from datasets import load_dataset

dataset = load_dataset("yahma/alpaca-cleaned", split="train")

README.md: 0.00B [00:00, ?B/s]

alpaca_data_cleaned.json:   0%|          | 0.00/44.3M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/51760 [00:00<?, ? examples/s]

In [ ]:
dataset[100]

{'output': "I'm sorry, but I don't have enough contextual information about the Epson F7100 to answer that question.",
 'input': '',
 'instruction': "Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\nPreviously, the use of dye-sub printing was limited to industrial or high-end commercial printing. Dye-sub photo printing has been used in medical imaging, graphic arts proofing, security, and broadcast-related applications. Today, it is extremely popular in event photography and photo booths or kiosks that require high-speed, on-demand printing.\n\nAlps Electric produced the first quality dye-sub printers for home consumers in the $500–$1,000 price range, bringing dye-sublimation technology within the reach of a wider audience. (These models were, however, not true page printers, since they used a narrow printhead that swept across the page, like most inkjet printers.) Now t

# **Load model**

In [ ]:
max_seq_length = 2048

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-1B-Instruct",
    max_seq_length=max_seq_length,
    dtype=None, # mô hình sẽ tự đi chọn dtype phù hợp
    load_in_4bit=False # Để mô hình dùng LoRA
)

==((====))==  Unsloth 2025.11.6: Fast Llama patching. Transformers: 4.56.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

In [ ]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 2048, padding_idx=128004)
    (layers): ModuleList(
      (0-15): 16 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=512, bias=False)
          (v_proj): Linear(in_features=2048, out_features=512, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (up_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (down_proj): Linear(in_features=8192, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
 

+ **gate_proj**: nó biến đổi đầu vào, và đầu ra sẽ đi qua gate => up_proj dùng phép nhân element-wise
+ **up_proj**: Nó sẽ tăng kích thước lên một không gian mới có kích thước lớn hơn (từ 2048 lên 8192)
+ **down_proj**: áp dụng layer act_fn để trở về kích thước ban đầu (8192 xuống còn 2048)

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 8, # Rank của ma trận LoRA, giá trị càng lớn, khả năng biểu diễn càng tốt nhưng tăng số lượng tham số => tốn tài nguyên
    # Xác định các layer áp dụng LoRA => ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"]
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha=16, # Hệ số scaling cho ma trận LoRA (thường gấp đôi r)
    lora_dropout=0,
    bias="none", # Không sử dụng bias trong các layer LoRA
    use_gradient_checkpointing="unsloth",
    random_state=42,
    use_rslora=False,
    loftq_config=None # LoftQ một phương pháp lượng tử hoá kết hợp với LoRA
)

Unsloth 2025.11.6 patched 16 layers with 16 QKV layers, 16 O layers and 16 MLP layers.


# **Tiền xử lý dữ liệu huấn luyện mô hình**

Tuân theo cấu trúc của alpaca_prompt, các dữ liệu sẽ được format lại theo cấu trúc của alpaca_prompt và thêm EOS_TOKEN, từ đó tạo thành dữ liệu huấn luyện.

In [ ]:
alpaca_prompt = """
Below is an instruction that describes a task, paired with an input that provides further context.
 Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}
"""

In [ ]:
EOS_TOKEN = tokenizer.eos_token

In [ ]:
def format_prompts_func(sentences):
    instructions = sentences["instruction"]
    inputs = sentences["input"]
    outputs = sentences["output"]
    texts = []

    for instruction, input, output in zip(instructions, inputs, outputs):
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return {"text": texts}

In [ ]:
dataset = dataset.map(format_prompts_func, batched=True)

Map:   0%|          | 0/51760 [00:00<?, ? examples/s]

# **Huấn luyện model**

In [ ]:
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,

        warmup_steps = 5,
        max_steps = 60,

        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none",
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/51760 [00:00<?, ? examples/s]

In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 51,760 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 5,636,096 of 1,241,450,496 (0.45% trained)
/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:1044: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. Starting in PyTorch 2.9, calling checkpoint without use_reentrant will raise an exception. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:1044: UserWarning: torch.utils.checkpoint: the use_reentrant parameter sh

Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,1.903600
2,2.466700
3,2.022200
4,2.306400
5,2.050700
6,1.910600
7,1.504100
8,1.682900
9,1.521600
10,1.481500


/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:1044: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. Starting in PyTorch 2.9, calling checkpoint without use_reentrant will raise an exception. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:1044: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. Starting in PyTorch 2.9, calling checkpoint without use_reentrant will raise an exception. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


In [ ]:
messages = [
    {"role": "user", "content": "During the last member meeting, create a list of 5 ideas to improve the club."}
]

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "/content/outputs/checkpoint-60",
    max_seq_length=max_seq_length,
    dtype=None,
    load_in_4bit=False
)

==((====))==  Unsloth 2025.11.6: Fast Llama patching. Transformers: 4.56.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [ ]:
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,
    return_tensors="pt"
).to("cuda")

In [ ]:
from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 1024,
                   use_cache = True, temperature = 1.5, min_p = 0.1)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Here are five potential ideas to improve your club:

1. **Online Club Platform Upgrade**: Create a modern and user-friendly online platform that makes it easy for members to engage with each other, access club resources, and share information. Consider adding features such as group chats, discussions, and social media integration to improve communication and participation.

2. **Enhanced Member Recognition**: Develop a system that allows members to nominate and recognize one another for their contributions to the club, such as sharing tips or best practices. This can foster a sense of camaraderie and motivation among members.

3. **Training and Skill-Sharing**: Organize regular training sessions or workshops where members can learn new skills or attend discussions on topics relevant to their profession. This can help keep the club active and relevant, while also providing members with valuable knowledge and expertise.

4. **Collaborative Projects**: Encourage members to collaborate on 